In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Colab Notebooks/nn/"

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Colab Notebooks/nn/"
%cd /content/gdrive/MyDrive/Colab\ Notebooks/nn
!pwd

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler

filename = "all_stocks_5yr.csv"
original_data = pd.read_csv(filename)

In [ ]:
original_data.describe()
original_data.head()

In [ ]:
from matplotlib.dates import AutoDateFormatter, AutoDateLocator, date2num

len(original_data['Name'].unique())

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def generate_series(data, value_num):
    min = data.min()["close"]
    max = data.max()["close"]
    close = (data['close'] - min)/(max- min)
    tsg = tf.keras.preprocessing.sequence.TimeseriesGenerator(np.asanyarray(close), np.asanyarray(close),
                              length=value_num,
                              batch_size=len(close))
    return tsg

def create_model(n):
    m = keras.Sequential()
    m.add(layers.Dense(64, activation='relu', input_shape=(n,)))
    m.add(layers.Dense(64, activation='relu'))
    m.add(layers.Dense(1))
    return m

def train_models(model_options):
    models = {}

    for model_opt in model_options:
        print('Using {} inputs'.format(model_opt["name"]))

        window_size = model_opt['window_size']
        m = create_model(window_size)
        print('Training...')
        epochs = model_opt["epochs"]
        m.compile(optimizer='adam', loss='mse') 

        data = model_opt['data']

        train_generator = generate_series(data[:-300], window_size)
        val_generator = generate_series(data[-300:-100], window_size)

        h = m.fit(train_generator,
                  epochs=epochs,
                  validation_data=val_generator)


        model_opt_to_save = dict(model_opt)
        del model_opt_to_save['data']
        model_info = {'model': m, 'history': h.history, 'inputs': window_size, "options": model_opt_to_save}
        models[window_size] = model_info
    return models

In [ ]:
def filter_stocks(all_data, names):
  filterd_data = all_data[all_data["Name"] == names[0]]
  for name in names[1:]:
    filterd_data = filterd_data.append(all_data[all_data["Name"] == name])
  return filterd_data

first_stock = filter_stocks(original_data, ["AAL"])

In [ ]:
import pickle
model_options = [
                 {"name": 'one_stock_15', "data": first_stock, "window_size": 15, "epochs": 250},
                 {"name": 'one_stock_20', "data": first_stock, "window_size": 20, "epochs": 250},
                 {"name": 'one_stock_25', "data": first_stock, "window_size": 25, "epochs": 250},
                 {"name": 'one_stock_35', "data": first_stock, "window_size": 35, "epochs": 250}
                 ]
trained_models = train_models(model_options)

In [ ]:
from tensorflow.keras.losses import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError

mse = MeanSquaredError()
mae = MeanAbsoluteError()
mape = MeanAbsolutePercentageError()

def convert_to_predict_input(data, offset, size, min, max):
    data_from_known = np.asarray(data['close'])
    normalized_data = (data_from_known - min)/(max- min)

    finalized = normalized_data[offset:offset+size]
    
    return np.reshape(finalized, (1,size))

model_stats = {}
test_data = original_data[original_data.Name == "GOOGL"]
# test_data = first_stock[-100:]
# test_data = first_stock[-300:]
# test_data = first_stock[5:]
mae_val = []
mape_val = []
mse_val = []
for k, v in trained_models.items():
    model = v['model']    
    window_size = v['inputs']
    initial_offset = 35 - window_size

    predicted_values = []
    min = test_data.min()["close"]
    max = test_data.max()["close"]
    for i in range(250):
        input_next = convert_to_predict_input(test_data[initial_offset+i:i+initial_offset+window_size], 0, window_size, min, max)
        predicted_value = model.predict(input_next)[0][0]
        predicted_values.append(predicted_value)


    predicted_values_inversed = np.asarray(predicted_values) * (max-min) + min

    fig, ax = plt.subplots()
    ax.set_title(v["options"]["name"])

    ax.plot(predicted_values_inversed, linestyle='dotted', color='blue')
    ax.plot(np.asarray(test_data[initial_offset + window_size:initial_offset+window_size+(i+1)]['close']), color='green')

    plt.legend(['Predicted price', 'Actual price'])
    real_stock_price = test_data[initial_offset + window_size:initial_offset+window_size+(i+1)]['close']
    mae_val.append(mae(real_stock_price, predicted_value).numpy())
    mape_val.append(mape(real_stock_price, predicted_value).numpy())
    mse_val.append(mse(real_stock_price, predicted_value).numpy())

    print('mae', mae_val[-1])
    print('mape', mape_val[-1])
    print('mse', mse_val[-1])

    plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_title("Mean squred error")

ax.plot(mse_val, linestyle='dotted', marker='o', color='blue')
plt.show()